In [1]:
import pandas as pd
import os
import numpy as np
import glob
from xgboost import XGBRegressor
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('input/optiver-realized-volatility-prediction/train.csv')
test = pd.read_csv('input/optiver-realized-volatility-prediction/test.csv')

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

In [ ]:
list_order_book_file_train = glob.glob('input/optiver-realized-volatility-prediction/book_train.parquet/*')

In [ ]:
def processing(folder_path):
    df_all=[]
    for file in tqdm(folder_path):
        stock_id = file.split('=')[1]
        df_book_data = pd.read_parquet(file)
        df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (df_book_data['bid_size1']+ df_book_data['ask_size1'])
        df_book_data['wap2'] =(df_book_data['bid_price2'] * df_book_data['ask_size2']+df_book_data['ask_price2'] * df_book_data['bid_size2'])  / (df_book_data['bid_size2']+ df_book_data['ask_size2'])
        df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
        df_book_data['log_return2'] = df_book_data.groupby(['time_id'])['wap2'].apply(log_return)
        df_book_data['row_id'] = df_book_data['time_id'].apply(lambda x:f'{stock_id}-{x}')
        
        df_book_data['price_diff'] = (df_book_data['ask_price1'] - df_book_data['bid_price1']) / ((df_book_data['ask_price1'] + df_book_data['bid_price1']) / 2)
        df_book_data['price_diff2'] = (df_book_data['ask_price2'] - df_book_data['bid_price2']) / ((df_book_data['ask_price2'] + df_book_data['bid_price2']) / 2)
        df_book_data['bid_diff'] = df_book_data['bid_price1'] - df_book_data['bid_price2']
        df_book_data['ask_diff'] = df_book_data['ask_price1'] - df_book_data['ask_price2']
        df_book_data['bid_ask_diff'] = abs(df_book_data['bid_diff'] - df_book_data['ask_diff'])
        df_book_data['total_volm'] = (df_book_data['ask_size1'] + df_book_data['ask_size2']) + (df_book_data['bid_size1'] + df_book_data['bid_size2'])
        df_book_data['volume_imbal'] = abs((df_book_data['ask_size1'] + df_book_data['ask_size2']) - (df_book_data['bid_size1'] + df_book_data['bid_size2']))
        
        create_feature_dict = {
        'wap': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'price_diff':[np.sum, np.mean, np.std],
        'bid_diff':[np.sum, np.mean, np.std],
        'ask_diff':[np.sum, np.mean, np.std],
        'total_volm':[np.sum, np.mean, np.std],
        'volume_imbal':[np.sum, np.mean, np.std]
        }
        
        def get_stats_window(seconds_in_bucket, add_suffix = False):
            # Group by the window
            df_feature = df_book_data[df_book_data['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
            # Rename columns joining suffix
            df_feature.columns = ['_'.join(col) for col in df_feature.columns]
            # Add a suffix to differentiate windows
            if add_suffix:
                df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
            return df_feature
    
        df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
        df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
        df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
        df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)
    
        df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
        df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
        df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
        # Drop unnecesary time_ids
        df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis = 1, inplace = True)
        stock_id = file.split('=')[1]
        df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
        df_feature.drop(['time_id_'], axis = 1, inplace = True)
        df_all.append(df_feature)
    return pd.concat(df_all)

In [ ]:
df_train = processing(list_order_book_file_train)

In [ ]:
list_order_trade_file_train = glob.glob('input/optiver-realized-volatility-prediction/trade_train.parquet/*')
#list_order_trade_file_train = list_order_trade_file_train[0:3]

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

# Function to preprocess trade data (for each stock id)
def trade_preprocessor(folder_path):
    df_all = []
    for file in tqdm(folder_path):
        df = pd.read_parquet(file)
        df['log_return'] = df.groupby('time_id')['price'].apply(log_return)

        # Dict for aggregations
        create_feature_dict = {
            'log_return':[realized_volatility],
            'seconds_in_bucket':[count_unique],
            'size':[np.sum],
            'order_count':[np.mean],
        }

        # Function to get group stats for different windows (seconds in bucket)
        def get_stats_window(seconds_in_bucket, add_suffix = False):
            # Group by the window
            df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
            # Rename columns joining suffix
            df_feature.columns = ['_'.join(col) for col in df_feature.columns]
            # Add a suffix to differentiate windows
            if add_suffix:
                df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
            return df_feature

        # Get the stats for different windows
        df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
        df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
        df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
        df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)

        # Merge all
        df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
        df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
        df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
        # Drop unnecesary time_ids
        df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis = 1, inplace = True)

        df_feature = df_feature.add_prefix('trade_')
        stock_id = file.split('=')[1]
        df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
        df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
        df_all.append(df_feature)
    return pd.concat(df_all)

In [ ]:
df_train_2 = trade_preprocessor(list_order_trade_file_train)

In [ ]:
df_train = df_train.merge(df_train_2,how='left',on=['row_id'])

In [ ]:
df_train[['stock_id','time_id']] = pd.DataFrame(df_train['row_id'].str.split('-').to_list(),index=df_train.index)

In [ ]:
# Get realized volatility columns
vol_cols = ['log_return_realized_volatility', 'log_return2_realized_volatility', 'log_return_realized_volatility_450', 'log_return2_realized_volatility_450', 
            'log_return_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return_realized_volatility_150', 'log_return2_realized_volatility_150', 
            'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_450', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_150']

# Group by the stock id
df_stock_id = df_train.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
# Rename columns joining suffix
df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
df_stock_id = df_stock_id.add_suffix('_' + 'stock')

# Group by the stock id
df_time_id = df_train.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
# Rename columns joining suffix
df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
df_time_id = df_time_id.add_suffix('_' + 'time')

# Merge with original dataframe
df_train = df_train.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
df_train = df_train.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
df_train.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)


In [6]:
def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w


def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe


def rmspe_xg(yhat, y):
    # y = y.values
    y = y.get_label()
    y = np.exp(y) - 1
    yhat = np.exp(yhat) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y - yhat)**2))
    return "rmspe", rmspe


In [ ]:
train['row_id'] = train['stock_id'].astype(str)+'-'+train['time_id'].astype(str)
df_train[['stock_id','time_id']] = df_train[['stock_id','time_id']].astype(int)
train = train.merge(df_train,how='left',on=['row_id','stock_id','time_id'])
columns = train.drop(columns=['stock_id','time_id','target','row_id']).columns
train[columns] = train[columns].astype(float)
train = train.fillna(0)

In [ ]:
train.to_csv('train.csv')

In [2]:
train = pd.read_csv('train.csv')
train = train.drop(columns=['Unnamed: 0'])

In [ ]:
train

In [3]:
columns = train.drop(columns=['stock_id','time_id','target','row_id']).columns
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns=['target']),train['target'],random_state=42,test_size = .025)

In [4]:
params = {'colsample_bytree': 0.7, 
          'learning_rate': 0.05, 
          'max_depth': 8, 
          'min_child_weight': 4, 
          'n_estimators': 10000, 
          'nthread': 4, 
          'objective': 'reg:squaredlogerror', 
          'subsample': 0.7, 
          'tree_method': 'gpu_hist'
          }

In [7]:
model = XGBRegressor(**params, feval=rmspe_xg)

In [8]:
model.fit(X_train[columns],y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7,
             feval=<function rmspe_xg at 0x000001B5505A70D0>, gamma=0, gpu_id=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=8,
             min_child_weight=4, missing=nan, monotone_constraints='()',
             n_estimators=10000, n_jobs=4, nthread=4, num_parallel_tree=1,
             objective='reg:squaredlogerror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [10]:
xg_preds = model.predict(X_test[columns])

In [ ]:
from sklearn.metrics import r2_score
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
R2 = round(r2_score(y_true = y_test, y_pred = xg_preds),3)
RMSPE = round(rmspe(y_true = y_test, y_pred = xg_preds),3)
print(f' R2 score: {R2}, RMSPE: {RMSPE}')

In [9]:
model.save_model('optiverv4.json')

C:\Users\conno\anaconda3\lib\site-packages\xgboost\sklearn.py:537: UserWarning: kwargs is not saved in Scikit-Learn meta.
  warnings.warn(str(k) + ' is not saved in Scikit-Learn meta.', UserWarning)


In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print(torch.cuda.get_device_name(0))
print()

Using device: cuda
GeForce RTX 2060 SUPER



In [55]:
#convert training date to tensors
X_nn = torch.FloatTensor(np.array(X_train[columns]))
X_nn_1 = X_train[['stock_id']].copy().values.astype(np.int64)
y_nn = torch.FloatTensor(np.array(y_train))

In [56]:
batch_size = 256

In [57]:
#model definition
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.emb = nn.Embedding(max(X_train['stock_id'])+1, 16)
        self.emb_drop = nn.Dropout(0.25)
        self.emb_bn = nn.BatchNorm1d(X_nn.shape[1])
        self.fc1 = nn.Linear(16+X_nn.shape[1],1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 32)
        self.fc7 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.25)
        self.bn1 = nn.BatchNorm1d(1024)
        self.bn2 = nn.BatchNorm1d(512)
        self.bn3 = nn.BatchNorm1d(256)
        self.bn4 = nn.BatchNorm1d(128)
        self.bn5 = nn.BatchNorm1d(64)
        self.bn6 = nn.BatchNorm1d(32)
        


    def forward(self, x_cat, x_cont):
        x1 = self.emb(x_cat)
        x1 = torch.flatten(x1, end_dim=1)
        x1 = self.emb_drop(x1)
        x2 = self.emb_bn(x_cont)
        x = torch.cat([x1, x2], 1)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = F.relu(self.bn4(self.fc4(x)))
        x = self.dropout(x)
        x = F.relu(self.bn5(self.fc5(x)))
        x = self.dropout(x)
        x = F.relu(self.bn6(self.fc6(x)))
        x = self.dropout(x)
        x = self.fc7(x)
        
      
        return x
net = Net().to(device)
print(net)

Net(
  (emb): Embedding(127, 16)
  (emb_drop): Dropout(p=0.25, inplace=False)
  (emb_bn): BatchNorm1d(228, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=244, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=32, bias=True)
  (fc7): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (bn1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [58]:
#reshape into tensor sequences for loading
seq = []
for i in range(0,len(X_nn)):
    seq.append(((X_nn_1[i],X_nn[i]),y_nn[i]))
    i = i + 1

In [59]:
#create trainloader
trainloader = torch.utils.data.DataLoader(seq, batch_size=batch_size,shuffle=True, num_workers=2)

In [60]:
#set loss function and optimizer
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=0.001)
steps = 30
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=5, mode = 'min',factor=.9)

# create a function (this my favorite choice)
def RMPSELoss(yhat,y):
    return torch.sqrt(torch.mean((((y-yhat)/y)**2)))
criterion = RMPSELoss

In [61]:
net.load_state_dict(torch.load('models/nn.pt'), strict=False)
net.train()
#model training
running_loss_min = 0.23
import time
start_time = time.time()
for epoch in tqdm(range(500)):  # loop over the dataset multiple times
    running_loss = 0.0
    running_val_loss = 0.0
    print(optimizer.param_groups[0]['lr'])
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        (cats,counts), preds = data
        # forward + backward + optimize
        outputs = net(cats.to(device),counts.to(device))
        loss = criterion(outputs.reshape(1,-1), preds.reshape(1,-1).to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    if (running_loss/(i + 1)) < running_loss_min:
        torch.save(net.state_dict(), 'models/nn.pt')
        running_loss_min = (running_loss/(i + 1))
        print('Best Loss')
    scheduler.step(running_loss)
    print(running_loss/(i + 1))
print('Finished Training')
print("time",(time.time()-start_time))

0.001
0.25068743406559
0.001
0.25013423991028216
0.001
0.25002367022097327
0.001
0.24931999907864205
0.001
0.2501960990259668
0.001
0.25412102485921423
0.001
0.2507967243616263
0.001
0.24908102403145232
0.001
0.25071049149682795
0.001
0.2504851904445792
0.001
0.2512850871941159
0.001
0.2496652092643292
0.001
0.25044456659616715
0.001
0.24957533088607334
0.0009000000000000001
0.2484195341368638
0.0009000000000000001
0.24652763324786284
0.0009000000000000001
0.247255380763564
0.0009000000000000001
0.24811417536497699
0.0009000000000000001
0.24666177461350125
0.0009000000000000001
0.24752713767230292
0.0009000000000000001
0.24528895344954518
0.0009000000000000001
0.24617102404775934
0.0009000000000000001
0.2466054292831164
0.0009000000000000001
0.24542382880224592
0.0009000000000000001
0.2466572218163069
0.0009000000000000001
0.2443500791780076
0.0009000000000000001
0.24612039988780693
0.0009000000000000001
0.2528683220833139
0.0009000000000000001
0.2527355813994694
0.0009000000000000001


KeyboardInterrupt: 

In [ ]:
data

In [44]:
#convert training date to tensors
X_test_nn = torch.FloatTensor(np.array(X_test[columns]))
X_test_nn_1 = X_test[['stock_id']].copy().values.astype(np.int64)
y_test_nn = torch.FloatTensor(np.array(y_test))

In [45]:
#reshape into tensor sequences for loading
testseq = []
for i in range(0,len(X_test_nn)):
    testseq.append(((X_test_nn_1[i],X_test_nn[i]),y_test_nn[i]))
    i = i + 1   

In [46]:
len(testseq)

10724

In [47]:
#create trainloader
testloader = torch.utils.data.DataLoader(testseq, batch_size=len(testseq),shuffle=False, num_workers=1)

In [62]:
net.load_state_dict(torch.load('models/nn.pt'), strict=False)
net_eval = net.eval().cpu()
nn_preds = []
for (x_cat, x_cont), y_ in tqdm(testloader):
    nn_preds = net_eval(x_cat,x_cont).detach().cpu().numpy().flatten()

In [63]:
from sklearn.metrics import r2_score
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
R2 = round(r2_score(y_true = y_test, y_pred = nn_preds),3)
RMSPE = round(rmspe(y_true = y_test, y_pred = nn_preds),3)
print(f' R2 score: {R2}, RMSPE: {RMSPE}')

 R2 score: 0.843, RMSPE: 0.207


In [ ]:
torch.save(net.state_dict(), 'models/nn.pt')

In [64]:
xg_preds

array([0.00644385, 0.00226229, 0.00285133, ..., 0.00199356, 0.00360787,
       0.00332407], dtype=float32)

In [65]:
nn_preds

array([0.00567519, 0.00241666, 0.00253724, ..., 0.00194683, 0.00319025,
       0.00320705], dtype=float32)

In [66]:
comb_preds = (xg_preds+nn_preds)/2

In [67]:
R2 = round(r2_score(y_true = y_test, y_pred = comb_preds),3)
RMSPE = round(rmspe(y_true = y_test, y_pred = comb_preds),3)
print(f' R2 score: {R2}, RMSPE: {RMSPE}')

 R2 score: 0.88, RMSPE: 0.204
